[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=PracticalRunMarketmakerSpot.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=PracticalRunMarketmakerSpot.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Practical: Run Marketmaker On Spot

This notebook shows is very similar to the regular [Practical: Run Marketmaker example](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=PracticalRunMarketmaker.ipynb). The only difference is this one runs on a Spot market instead of a Perp market.

It contains the following sections:
1. This section contains all the code that is identical to the [Practical: Run Marketmaker example](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=PracticalRunMarketmaker.ipynb).
2. This section contains the code that is new to this notebook, specifically for working with a Spot market. It contains one method - `build_spot_model_state()` - which loads Spot market data.
3. This final section runs the marketmaker.

## 1. Identical Code

This code is all taken from the [Practical: Run Marketmaker example](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=PracticalRunMarketmaker.ipynb) with no changes. If you followed along with that Practical then you have already seen all this code.

In [ ]:
import logging
import mango
import mango.marketmaking
import time

from datetime import timedelta
from decimal import Decimal

from mango.marketmaking.orderchain.chain import Chain
from mango.marketmaking.orderchain.biasquoteonpositionelement import BiasQuoteOnPositionElement
from mango.marketmaking.orderchain.ratioselement import RatiosElement
from mango.marketmaking.orderchain.roundtolotsizeelement import RoundToLotSizeElement

# Set logging so messages show up.
logging.getLogger().setLevel(logging.INFO)


def build_context():
    return mango.ContextBuilder.build(cluster_name="devnet")


def build_order_chain():
    order_type = mango.OrderType.POST_ONLY
    spread = Decimal("0.005")  # 0.5%
    position_size = Decimal("0.05")  # 5%
    bias = [Decimal("0.0001")]

    ratios_element = RatiosElement(order_type, None, 20, [spread], [position_size], False)
    bias_element = BiasQuoteOnPositionElement(bias)
    round_to_lot_size_element = RoundToLotSizeElement()

    chain = Chain([ratios_element, bias_element, round_to_lot_size_element])

    return chain


def build_order_reconciler():
    return mango.marketmaking.ToleranceOrderReconciler(Decimal("0.001"), Decimal("0.001"), timedelta(seconds=5))


def build_oracle(context, market):
    oracle_source_name = "pyth"
    oracle_provider = mango.create_oracle_provider(context, oracle_source_name)
    oracle = oracle_provider.oracle_for_market(context, market)
    if oracle is None:
        raise Exception(f"Could not find oracle for market {market.symbol} from provider {oracle_source_name}.")
    return oracle


def build_marketmaker(context, wallet, account, market):
    chain = build_order_chain()
    order_reconciler = build_order_reconciler()
    instruction_builder = mango.instruction_builder(context, wallet, account, market.symbol, False)
    market_maker = mango.marketmaking.MarketMaker(wallet, market, instruction_builder, chain, order_reconciler, Decimal(0))
    return market_maker


## 2. Build the Spot `ModelState`

In the previous [Practical: Run Marketmaker example](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=PracticalRunMarketmaker.ipynb) the `build_model_state()` function handled building the `ModelState` for a Perp market.

There are some important differences between Spot markets and Perp markets, so this section defines a `build_spot_model_state()` function to perform the equivalent actions to build the `ModelState` for a Spot market.

In [ ]:
def build_spot_model_state(context, account, market, oracle, group_address, cache_address):
    # Build a list of addresses and use `load_multiple()` to fetch them all in one go.
    addresses = [
        group_address,
        cache_address,
        account.address,
        market.bids_address,
        market.asks_address,
        market.event_queue_address,
        *list([oo for oo in account.spot_open_orders if oo is not None])
    ]
    account_infos = mango.AccountInfo.load_multiple(context, addresses)
    group = mango.Group.parse_with_context(context, account_infos[0])
    cache = mango.Cache.parse(account_infos[1])
    account = mango.Account.parse(account_infos[2], group, cache)

    spot_open_orders_account_infos_by_address = {
        str(account_info.address): account_info for account_info in account_infos[6:]}

    all_open_orders = {}
    for slot in account.slots:
        if slot.spot_open_orders is not None and str(slot.spot_open_orders) in spot_open_orders_account_infos_by_address:
            account_info: mango.AccountInfo = spot_open_orders_account_infos_by_address[str(slot.spot_open_orders)]
            open_orders: mango.OpenOrders = mango.OpenOrders.parse(
                account_info,
                slot.base_token_bank.token,
                account.shared_quote_token)
            all_open_orders[str(slot.spot_open_orders)] = open_orders

    group_slot = group.slot_by_spot_market_address(market.address)
    open_orders_address = account.spot_open_orders_by_index[group_slot.index]
    placed_orders_container: mango.PlacedOrdersContainer = all_open_orders[str(open_orders_address)]

    # Spot markets don't accrue MNGO liquidity incentives
    mngo = mango.token(context, "MNGO")
    mngo_accrued = mango.InstrumentValue(mngo, Decimal(0))

    base_value = mango.InstrumentValue.find_by_symbol(account.net_values, market.base.symbol)
    quote_value = account.shared_quote.net_value

    event_queue = mango.SerumEventQueue.parse(account_infos[5])

    frame = account.to_dataframe(group, all_open_orders, cache)
    available_collateral = account.init_health(frame)
    inventory = mango.Inventory(mango.InventorySource.ACCOUNT,
                                mngo_accrued,
                                available_collateral,
                                base_value,
                                quote_value)

    orderbook = market.parse_account_infos_to_orderbook(account_infos[3], account_infos[4])

    price = oracle.fetch_price(context)

    return mango.ModelState(open_orders_address,
                            market,
                            mango.ManualUpdateWatcher(group),
                            mango.ManualUpdateWatcher(account),
                            mango.ManualUpdateWatcher(price),
                            mango.ManualUpdateWatcher(placed_orders_container),
                            mango.ManualUpdateWatcher(inventory),
                            mango.ManualUpdateWatcher(orderbook),
                            mango.ManualUpdateWatcher(event_queue))

## 8. Run the `MarketMaker`

This code is also _nearly_ identical to the [Practical: Run Marketmaker example](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=PracticalRunMarketmaker.ipynb). The differences are:

* It loads market SOL/USDC instead of SOL-PERP
* It uses a `SpotCollateralCalculator` instead of a `PerpCollateralCalculator`
* It calls our new `build_spot_model_state()` to build the `ModelState`

### Caveat

Please bear in mind that the code below uses a shared Solana `Keypair` and a shared Mango `Account`. If you're going to run this example more than once or twice, it might be better for you to set up a fresh devnet `Keypair` and `Account` for you to use - it's easy and free, and it means other people running this example won't accidentally cancel your orders or cause confusion.

In [ ]:
# Use our hard-coded devnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = mango.Wallet(bytes([67,218,68,118,140,171,228,222,8,29,48,61,255,114,49,226,239,89,151,110,29,136,149,118,97,189,163,8,23,88,246,35,187,241,107,226,47,155,40,162,3,222,98,203,176,230,34,49,45,8,253,77,136,241,34,4,80,227,234,174,103,11,124,146]))

# Specify the market we're going to use
market_symbol = "SOL/USDC"

# Configure how long to pause between pulses
pause_seconds = 30

# Create a 'devnet' Context
context = build_context()

# Load the wallet's account
group = mango.Group.load(context)
accounts = mango.Account.load_all_for_owner(context, wallet.address, group)
account = accounts[0]

# Load the market
market = mango.market(context, market_symbol)

oracle = build_oracle(context, market)
marketmaker = build_marketmaker(context, wallet, account, market)

stop_requested = False
while not stop_requested:
    try:
        model_state = build_spot_model_state(context, account, market, oracle, group.address, group.cache)
        marketmaker.pulse(context, model_state)

        # Wait and hope for fills.
        print(f"Pausing for {pause_seconds} seconds.\n")
        time.sleep(pause_seconds)
    except KeyboardInterrupt:
        stop_requested = True
    except Exception as exception:
        print(f"Continuing after problem running market-making iteration: {exception}")

market_operations = mango.operations(context, wallet, account, market.symbol, dry_run=False)
for order in market_operations.load_my_orders():
    market_operations.cancel_order(order, ok_if_missing=True)

context.dispose()
print("Example complete")
